## Purpose of this notebook

- Examine similarities and differences between different raters' reliability report grades
- Calculate Cohen's kappa between pairs of raters
- Identify and print reports where raters strongly disagree (grade of 2 vs grade of 0)

## How to use this notebook

- Run each of the cells in order. Make sure you run Cell 01 first.
- Cell 02 can be used to see how many reports have been graded since a date (YYYY-MM-DD format)
- Cells 03-04 get the proc_ord_id values unique to each report and the names of the persons who have graded reliability reports.
- Cell 05 can be used to examine the reports a pair of graders disagree on
- Cells 06 will release examined reports back to a specified grader for regrading
- Cell 07 can be used to examine and regrade reports marked with a -1 flag

In [ ]:
# Cell 01: load libraries
from reliabilityLib import *
from reportMarkingFunctions import *
from google.cloud import bigquery # SQL table interface on Arcus
import pandas
import numpy
import matplotlib.pyplot as plt

client = bigquery.Client()

## Evolution of SLIP over time

In [ ]:
# Cell 02:
getGradeCountsSinceDate("2024-05-01")

## Grader agreement on reliability reports

In [ ]:
# Cell 03: Get the list of proc_ord_id values used to identify the reliability reports
procIds = getReliabilityProcOrdIds()

In [ ]:
# Cell 04: Compare the reliability reports for the users we want to evaluate
graders = ['Jenna Schabdach', 
           'Megan M. Himes', 
           'Naomi Shifman', 
           'Alexa DeJean',
           'Julia Katowitz',
           'Matt Buczek',
           'Shivaram Karandikar', 
           'Shreya Gudapati']

# Metric options: "disagreement", "kappa", "kappa2vAll", "kappa0vAll"
df = calculateMetricForGraders(graders, "kappa")
print()
print(df)

In [ ]:
# Cell 05: 
# This is the cell where you can look at the disagreement reports for each pair of users
# User 1: Naomi
# User 2: Jenna
grader1 = "Jenna Schabdach"
grader2 = "Naomi Shifman"
# procIds = getReliabilityProcOrdIds()
# grades1 = getReportsForUser(grader1, procIds)
# grades2 = getReportsForUser(grader2, procIds)
# disagreement = identifyDisagreementReports(grades1, grades2)

# printDisagreementReports(disagreement, grades1, grades2)

In [ ]:
# Cell 06: release a set of your reports back into your queue 
# ONLY USE THIS IF YOU'RE CERTAIN
# releaseReports(grader2, disagreement)

## Examine flagged reports

In [ ]:
# Cell 07: regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
skippedGrader = "" # "Naomi Shifman"
regradeSkippedReports(client, skippedGrader)

## For clinician review only

In [ ]:
# Cell 08: clinician regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
regradeSkippedReports(client, flag=-2)